In [ ]:
from pyserini.search.faiss import FaissSearcher
from pyserini.search.lucene import LuceneSearcher
from pyserini.encode import AutoQueryEncoder
from pyserini.search import get_topics, get_qrels
from tqdm import tqdm

### Initialize Contriever Index and Query Encoder
We use [Pyserini](https://github.com/castorini/pyserini) as the search interface for the experiment. Please following the guidance in Pyserini to create Contriever index using the checkpoint from original Contriever work.

In [2]:
query_encoder = AutoQueryEncoder(encoder_dir='facebook/contriever', pooling='mean')
searcher = FaissSearcher('contriever_msmarco_index/', query_encoder)
corpus = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Load query and judegments for dl19-passage dataset

In [5]:
topics = get_topics('dl19-passage')
qrels = get_qrels('dl19-passage')

## Run Contriever

In [6]:
with open('dl19-contriever-top1000-trec', 'w')  as f:
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            hits = searcher.search(query, k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

100%|█████████████████████████████████████████████████████| 43/43 [04:28<00:00,  6.25s/it]


In [8]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19-contriever-top1000-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19-contriever-top1000-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19-contriever-top1000-trec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/xueguang/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/xueguang/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/home/xueguang/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19-contriever-top1000-trec']
Results:
map                   	all	0.2399
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the envi

In [10]:
import openai
import os
from openai.error import RateLimitError
def call_codex_read_api(prompt: str):
    def parse_api_result(result):
        to_return = []
        for idx, g in enumerate(result['choices']):
            text = g['text']
            logprob = sum(g['logprobs']['token_logprobs'])
            to_return.append((text, logprob))
        res = [r[0] for r in sorted(to_return, key=lambda tup: tup[1], reverse=True)]
        return res
    result = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        api_key=os.getenv()['OPENAI'],
        max_tokens=512,
        temperature=0.7,
        top_p=1,
        n=8,
        stop=['\n\n\n'],
        logprobs=1
    )
    return parse_api_result(result)

## Run HyDE

In [11]:
from time import sleep
import numpy as np
import json
from tqdm import tqdm
with open('hyde-dl19-contriever-gpt3-top1000-8rep-trec', 'w') as f, open('hyde-dl19-gpt3-gen.jsonl', 'w') as fgen:
    for qid in tqdm(topics):
        if qid in qrels:
            query = topics[qid]['title']
            print(query)
            prompt = f"""Please write a passage to answer the question
Question: {query}
Passage:"""
            get_result = False
            while not get_result:
                try:
                    contexts = [c.strip() for c in call_codex_read_api(prompt)] + [query]
                    all_emb_c = []
                    for c in contexts:
                        c_emb = query_encoder.encode(c)
                        all_emb_c.append(np.array(c_emb))
                    all_emb_c = np.array(all_emb_c)
                    avg_emb_c = np.mean(all_emb_c, axis=0)
                    avg_emb_c = avg_emb_c.reshape((1, len(avg_emb_c)))
                    fgen.write(json.dumps({'query_id': qid, 'query': query, 'contexts': contexts})+'\n')
                    get_result = True
                except:
                    sleep(1)
            print(contexts)
            hits = searcher.search(avg_emb_c, k=1000)
            rank = 0
            for hit in hits:
                rank += 1
                f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')


  0%|                                                              | 0/43 [00:00<?, ?it/s]

how long is life cycle of flea
['The life cycle of a flea typically lasts 8 to 12 days. It begins with an egg, which hatches into larvae and then pupates. After that, the adult flea emerges and begins to feed on the host’s blood before reproducing and laying eggs. The eggs then hatch and the cycle begins anew. In optimal conditions, a flea can complete its life cycle in as little as two weeks. However, under unfavorable conditions, the cycle can take much longer, sometimes up to a year.', 'The life cycle of a flea is relatively short, lasting between two and three weeks. It begins with the egg stage, where a female flea will lay up to 50 eggs at a time. These eggs will hatch within two to five days and the larvae will feed on organic debris in the environment, such as bits of hair or skin flakes. After two to three weeks, the larvae will spin a cocoon and enter the pupal stage. During this time, the flea will develop wings and other characteristics of an adult flea. After about a week,


  2%|█▎                                                    | 1/43 [00:11<08:15, 11.81s/it]

cost of interior concrete flooring
['The cost of interior concrete flooring depends on a variety of factors, such as the size of the area to be covered, the type of concrete used, and the complexity of the job. Generally, a basic interior concrete floor will cost between $4 and $8 per square foot. The cost may increase if the job is more complex, or if extra labor or materials are needed. Additionally, if the concrete needs to be stained or sealed, there will be additional costs associated with those services. Ultimately, the cost of interior concrete flooring can vary greatly depending on the specifics of the job.', 'Interior concrete flooring is a popular choice for many homeowners and businesses due to its durability and low cost. The cost of interior concrete flooring depends on several factors, including the size of the area to be covered, the complexity of the installation, and the type of finish desired. Generally, basic interior concrete flooring can be installed for as little 


  5%|██▌                                                   | 2/43 [00:23<07:49, 11.45s/it]

...


 95%|██████████████████████████████████████████████████▌  | 41/43 [07:29<00:21, 10.85s/it]

who formed the commonwealth of independent states
['The Commonwealth of Independent States (CIS) was formed on 8 December 1991 by the leaders of Russia, Ukraine, and Belarus. The three leaders, Boris Yeltsin of Russia, Leonid Kravchuk of Ukraine, and Stanislav Shushkevich of Belarus, signed a treaty that created the Commonwealth of Independent States. The treaty was ratified by the three countries in January 1992, and other former Soviet republics joined the Commonwealth in the following months. The CIS is a regional intergovernmental organization that serves as a platform for the member states to coordinate their economic, trade, and security policies.', 'The Commonwealth of Independent States (CIS) was formed on December 8, 1991, when eight of the fifteen Soviet Republics declared their independence and signed the Alma-Ata Protocol. The members of the CIS are Armenia, Belarus, Kazakhstan, Kyrgyzstan, Moldova, Russia, Tajikistan, Turkmenistan, Ukraine, and Uzbekistan. Azerbaijan, Geor


 98%|███████████████████████████████████████████████████▊ | 42/43 [07:41<00:11, 11.02s/it]

define visceral?
['Visceral refers to the internal organs of the body, such as the heart, lungs, stomach and intestines. It is also used to refer to emotions and reactions that are instinctive and intuitive, rather than intellectual or reasoned. In this sense, visceral emotions are those that are felt deeply and strongly, beyond the realm of conscious thought.', 'Visceral is an adjective used to describe reactions or emotions that are strong and deeply felt. It typically refers to reactions that are instinctive and powerful, often linked to a physical response in the body. Visceral reactions can be positive, such as joy, or negative, such as fear or anger. They can also be related to a feeling of disgust or revulsion.', "Visceral is an adjective used to describe emotions and reactions that are deep, instinctive, and powerful. It is often used to refer to an immediate and intense reaction, such as fear or anger. Visceral reactions often take place without conscious thought and stem from

100%|█████████████████████████████████████████████████████| 43/43 [07:50<00:00, 10.94s/it]


In [12]:
!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage hyde-dl19-contriever-gpt3-top1000-8rep-trec


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/xueguang/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/xueguang/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/home/xueguang/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19-contriever-gpt3-top1000-8rep-trec']
Results:
map                   	all	0.4039
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly se